# Pandas Multiple or Heirarchical Indexing

In [1]:
import pandas as pd 
import numpy as np 

## The Bad Way

In [40]:
index = [('California', 2010), ('California', 2020),
         ('New York', 2010), ('New York', 2020),
         ('Texas', 2010), ('Texas', 2020)]
populations = [37253956, 39538223,
               19378102, 20201249,
               25145561, 29145505]
pop = pd.Series(populations, index=index)
pop

(California, 2010)    37253956
(California, 2020)    39538223
(New York, 2010)      19378102
(New York, 2020)      20201249
(Texas, 2010)         25145561
(Texas, 2020)         29145505
dtype: int64

## Better way

In [42]:
index = pd.MultiIndex.from_tuples(index)
pop = pop.reindex(index)
pop

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [44]:
pop[:, 2020]

California    39538223
New York      20201249
Texas         29145505
dtype: int64

## Multi-index as extra dimension

In [7]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


> Convert Multiply indexed `Series` into a conventionally indexed `DataFrame`.

In [8]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Add a column

In [9]:
pop_df = pd.DataFrame({
    'total': pop,
    'under18': [9284094, 8898092,
                4318033, 4181528,
                6879014, 7432474]
})
pop_df

total  under18
California 2000  33871648  9284094
           2010  37253956  8898092
New York   2000  18976457  4318033
           2010  19378102  4181528
Texas      2000  20851820  6879014
           2010  25145561  7432474

In [10]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.274096
            2010    0.238850
New York    2000    0.227547
            2010    0.215786
Texas       2000    0.329900
            2010    0.295578
dtype: float64

In [11]:
f_u18.unstack()

,2000,2010
California,0.274096,0.238850
New York,0.227547,0.215786
Texas,0.329900,0.295578


# Multiindex Creation

In [12]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a','a','b','b'],[1,2,1,2]], 
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.944187  0.504185
  2  0.739919  0.945495
b 1  0.013960  0.811076
  2  0.090264  0.940907

### A dictionary with tuples as keys

In [31]:
data = {('California', 2010): 37253956,
        ('California', 2020): 39538223,
        ('New York', 2010): 19378102,
        ('New York', 2020): 20201249,
        ('Texas', 2010): 25145561,
        ('Texas', 2020): 29145505}
pd.Series(data)

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

### Explicit multiindex constructors

In [32]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [33]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [34]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [35]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

## Level Names

In [47]:
pop.index.names = ['state', 'year']
pop

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

## Multiindex for Columns

In [48]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

In [49]:
data = np.round(np.random.randn(4,6), 1)
data[:,::2] *= 10

data += 37
data

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  34.8  42.0  37.6  41.0  36.5
     2      32.0  36.8  49.0  36.4  47.0  37.4
2014 1      30.0  38.2  46.0  37.7  26.0  38.3
     2      50.0  37.0  33.0  38.3  46.0  38.1

In [50]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      42.0  37.6
     2      49.0  36.4
2014 1      46.0  37.7
     2      33.0  38.3

In [50]:
health_data.loc[2013]

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
visit                                      
1        27.0  37.9  29.0  38.2  34.0  35.3
2        49.0  36.7  38.0  38.6  29.0  36.2

## Multiply indexed Series

In [51]:
pop

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

>Index with multiple terms

In [52]:
pop['California', 2010]

37253956

>Partial index returns a series

In [53]:
pop['California']

year
2010    37253956
2020    39538223
dtype: int64

>Partial slicing requires it be sorted

In [54]:
pop.loc['California':'New York']

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
dtype: int64

In [55]:
pop[:, 2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [56]:
pop[pop > 22000000]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

>Fancy indexing

In [57]:
pop[['California', 'Texas']]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

## Mutliply indexed DataFrames

In [51]:
health_data['Guido','HR']

year  visit
2013  1        29.0
      2        38.0
2014  1        33.0
      2        51.0
Name: (Guido, HR), dtype: float64

In [59]:
health_data.iloc[:2]

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  37.9  29.0  38.2  34.0  35.3
     2      49.0  36.7  38.0  38.6  29.0  36.2

In [62]:
health_data.iloc[:2,2:3]

subject    Guido
type          HR
year visit      
2013 1      29.0
     2      38.0

In [63]:
# Fail 
# health_data.loc[(:, 1), (:, 'HR')]

In [64]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,27.0,29.0,34.0
2014,1,48.0,33.0,34.0


## Rearranging multi-indexes

### Sorted and unsorted indices

>Unsorted: results in error if sliced

In [60]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.138268
      2      0.540345
c     1      0.191939
      2      0.648918
b     1      0.340233
      2      0.817503
dtype: float64

In [61]:
data.sort_index()

char  int
a     1      0.138268
      2      0.540345
b     1      0.340233
      2      0.817503
c     1      0.191939
      2      0.648918
dtype: float64

In [62]:
data.sort_index()['a':'b']

char  int
a     1      0.138268
      2      0.540345
b     1      0.340233
      2      0.817503
dtype: float64

## Stacking and unstacking

In [63]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2010,37253956,19378102,25145561
2020,39538223,20201249,29145505


In [64]:
pop.unstack(level=1)

year,2010,2020
state,,
California,37253956,39538223
New York,19378102,20201249
Texas,25145561,29145505


In [65]:
pop.unstack().stack()

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

# Index setting and resetting

In [71]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [67]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2010,37253956
1,California,2020,39538223
2,New York,2010,19378102
3,New York,2020,20201249
4,Texas,2010,25145561
5,Texas,2020,29145505


## Build MultiIndex from column values

In [68]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2010    37253956
           2020    39538223
New York   2010    19378102
           2020    20201249
Texas      2010    25145561
           2020    29145505